In [1]:
from collections import Counter
from collections import defaultdict
from array import array
import numpy as np
import heapq


tokens=[]
with open('../Assignment 4/corpus.txt',encoding='UTF-8') as f:
    for line in f:
        if not line.isspace():
            line=line.lower()
            line='<s> '+line.replace('.',' ').replace(',',' ').replace(':',' ').replace(';',' ').replace('!',' ').replace('?',' ').replace('(',' ').replace(')',' ').replace("-"," ").replace("' ",' ').replace('"',' ').replace("_"," ") +' </s>'
            items=line.split()
            tokens+=items    
tokenSet=set(tokens)
    


In [2]:
def makenGram(_tokens,_n):
    ngramlist=[]
    tokens=_tokens
    n=_n
    tokenlen=len(tokens)
    for counter,token in enumerate(tokens):
        single_ngramlist=[]
        for i in range(0,n):
            if(counter+n<=tokenlen):
                single_ngramlist.append(tokens[counter+i])
        if(len(single_ngramlist)==n):
            ngramlist.append(tuple(single_ngramlist))
    return ngramlist
        

In [3]:
def predict_next_word(Sentnce,Selection):
    
    
    top5=[]
    prob_sum=0
    if Selection=='unigram':
        unigram_Count=Counter()
        unigram_wordsList=makenGram(tokens,1)
        unigram_Count=Counter(unigram_wordsList)
        for key in unigram_Count:
            #print(key,' ',unigram_Count)
            prob_sum=(unigram_Count[key]/len(unigram_wordsList))
            unigram_Count[key]=prob_sum
        top5=heapq.nlargest(5, unigram_Count, key=unigram_Count.get)
        print(top5)
    if Selection=='bigram':
        word_list=(Sentnce.strip().split(' ')[len(Sentnce.strip().split(' '))-1],)
        bigram_Count=Counter()
        unigram_Count=Counter()
        bigram_wordsList=makenGram(tokens,2)
        bigram_Count=Counter(bigram_wordsList)
        unigram_wordsList=makenGram(tokens,1)
        unigram_Count=Counter(unigram_wordsList)
        for key in bigram_Count:
            prob_sum=(bigram_Count[key]/unigram_Count[key[0:1]])
            bigram_Count[key]=prob_sum

        prob_table = defaultdict(dict)
        for key in bigram_Count:
            p=bigram_Count[key]
            Bi=key[0:1]
            token=key[1]
            prob_table[Bi][token]=p
        top5=heapq.nlargest(5, prob_table[word_list], key=prob_table[word_list].get)
        print(top5)
    if Selection=='trigram':
        word_list=(Sentnce.strip().split(' ')[len(Sentnce.strip().split(' '))-2],Sentnce.strip().split(' ')[len(Sentnce.strip().split(' '))-1])
        trigram_Count=Counter()
        bigram_Count=Counter()
        trigram_wordsList=makenGram(tokens,3)
        trigram_Count=Counter(trigram_wordsList)
        bigram_wordsList=makenGram(tokens,2)
        bigram_Count=Counter(bigram_wordsList)
        for key in trigram_Count:
            prob_sum=(trigram_Count[key]/bigram_Count[key[0:2]])
            trigram_Count[key]=prob_sum

        prob_table = defaultdict(dict)
        for key in trigram_Count:
            p=trigram_Count[key]
            Bi=key[0:2]
            token=key[2]
            prob_table[Bi][token]=p
        top5=heapq.nlargest(5, prob_table[word_list], key=prob_table[word_list].get)

        print(top5)
    if len(top5)==0:
        predict_next_word(Sentnce,'unigram')


In [4]:
unigram_Count=Counter()
unigram_wordsList=makenGram(tokens,1)
unigram_Count=Counter(unigram_wordsList)
for key in unigram_Count:
    #print(key,' ',unigram_Count)
    prob_sum=(unigram_Count[key]/len(unigram_wordsList))
    unigram_Count[key]=prob_sum

#-_________________BiGram
bigram_Count=Counter()
unigram_Count=Counter()
bigram_wordsList=makenGram(tokens,2)
bigram_Count=Counter(bigram_wordsList)
unigram_wordsList=makenGram(tokens,1)
unigram_Count=Counter(unigram_wordsList)
for key in bigram_Count:
    prob_sum=(bigram_Count[key]/unigram_Count[key[0:1]])
    bigram_Count[key]=prob_sum

prob_table_bigram = defaultdict(dict)
for key in bigram_Count:
    p=bigram_Count[key]
    Bi=key[0:1]
    token=key[1]
    prob_table_bigram[Bi][token]=p
#________________________________________________________TRIGRAM


trigram_Count=Counter()
bigram_Count=Counter()
trigram_wordsList=makenGram(tokens,3)
trigram_Count=Counter(trigram_wordsList)
bigram_wordsList=makenGram(tokens,2)
bigram_Count=Counter(bigram_wordsList)
for key in trigram_Count:
    prob_sum=(trigram_Count[key]/bigram_Count[key[0:2]])
    trigram_Count[key]=prob_sum
prob_table_trigram = defaultdict(dict)
for key in trigram_Count:
    p=trigram_Count[key]
    Bi=key[0:2]
    token=key[2]
    prob_table_trigram[Bi][token]=p

In [5]:
def predict_next_word_using_initial_substring(Sentnce,Selection,initial_sub):
    top5=[]
    top5_using_substr=[]
    prob_sum=0
    if Selection=='unigram' or Selection=='1':
        top5=heapq.nlargest(50000, unigram_Count, key=unigram_Count.get)
        if initial_sub=='':
            top5_using_substr.append( [s for s in top5 if len(s[0])>=len(initial_sub)] )
        else:
            top5_using_substr.append( [s for s in top5 if len(s[0])>=len(initial_sub) and initial_sub in s[0][0:len(initial_sub)]])
#        for each_topk in top5:
#            cond=initial_sub in each_topk[0][0:len(initial_sub)+1]
#            print(cond,'\n',each_topk[0],'\n',each_topk[0][0:len(initial_sub)+1],'\n',len(initial_sub))
#            print('\n\n\n')
    if Selection=='bigram' or Selection=='2':
        word_list=(Sentnce.strip().split(' ')[len(Sentnce.strip().split(' '))-1],)
        top5=heapq.nlargest(500000, prob_table_bigram[word_list], key=prob_table_bigram[word_list].get)
        top5_using_substr.append( [s for s in top5 if len(s)>len(initial_sub) and initial_sub in s[0:len(initial_sub)]])
        #print(top5)
    if Selection=='trigram' or Selection=='3':
        word_list=(Sentnce.strip().split(' ')[len(Sentnce.strip().split(' '))-2],Sentnce.strip().split(' ')[len(Sentnce.strip().split(' '))-1])
        
        top5=heapq.nlargest(500000, prob_table_trigram[word_list], key=prob_table_trigram[word_list].get)
        top5_using_substr.append( [s for s in top5 if len(s)>len(initial_sub) and initial_sub in s[0:len(initial_sub)]])
        #print(top5)
    if len(top5_using_substr)==0:
        predict_next_word(Sentnce,'unigram')
    return top5_using_substr

In [6]:
#a=predict_next_word_using_initial_substring(_Sentnce,_Selection,'ک')

In [7]:
predited_words=predict_next_word_using_initial_substring('میں نے','3','')

In [8]:
predited_words=predict_next_word_using_initial_substring('اور','2','و')

In [9]:
predited_words=predict_next_word_using_initial_substring('میں ہوا','2','پ')

In [10]:
predited_words=predict_next_word_using_initial_substring('میں ہوا','1','ا')

In [11]:
predited_words

[[('اور',),
  ('اس',),
  ('ایک',),
  ('ان',),
  ('انہوں',),
  ('اپنے',),
  ('اپنی',),
  ('ایس',),
  ('افراد',),
  ('اب',),
  ('اسلام',),
  ('اے',),
  ('آئی',),
  ('ایم',),
  ('انہیں',),
  ('استعمال',),
  ('امریکی',),
  ('آباد',),
  ('اجلاس',),
  ('اگر',),
  ('اسے',),
  ('احمد',),
  ('اظہار',),
  ('ایف',),
  ('اہم',),
  ('اسی',),
  ('ایل',),
  ('او',),
  ('انھوں',),
  ('ادا',),
  ('اللہ',),
  ('الیکشن',),
  ('اعلان',),
  ('انصاف',),
  ('اسمبلی',),
  ('ایسے',),
  ('این',),
  ('اپنا',),
  ('اعظم',),
  ('آج',),
  ('ادارے',),
  ('اعلیٰ',),
  ('امریکا',),
  ('اضافہ',),
  ('اداروں',),
  ('آف',),
  ('اداکارہ',),
  ('امن',),
  ('ایسا',),
  ('اپریل',),
  ('ای',),
  ('احتجاج',),
  ('آپ',),
  ('انتخابات',),
  ('اقدامات',),
  ('الزام',),
  ('امریکہ',),
  ('انتظامیہ',),
  ('آنے',),
  ('انھیں',),
  ('اختیار',),
  ('ارب',),
  ('انتہائی',),
  ('اینڈ',),
  ('اسپتال',),
  ('افغانستان',),
  ('ایچ',),
  ('اداکار',),
  ('افسران',),
  ('آپریشن',),
  ('اقوام',),
  ('ابھی',),
  ('اسلامی',),
  ('ایسی',),

In [19]:
from tkinter import *


def ReplaceBrackets(text):
    text1=text
    chars = '()\n'
    for c in chars:
        text1 = text1.replace(c, "" )
    return text1


def radioButtonSelected():
    keyPressedInTextBox('')
    #print(var.get())

def keyPressedInTextBox(evt):
    radio_val= var.get()
    model_choice=str(radio_val)
    textbox_val=TextBox.get('1.0',END)
    initial_sub=''
    if model_choice=='1':
        textbox_val=ReplaceBrackets(textbox_val)
        initial_sub=textbox_val.split(' ')[len(textbox_val.split(' '))-1]
        #print(initial_sub)
    if model_choice=='2':
        textbox_val=ReplaceBrackets(textbox_val)
        initial_sub=textbox_val.split(' ')[len(textbox_val.split(' '))-1]
        #print(initial_sub)
    if model_choice=='3':
        textbox_val=ReplaceBrackets(textbox_val)
        initial_sub=textbox_val.split(' ')[len(textbox_val.split(' '))-1]
        #print(initial_sub)
        
    
    #print(textbox_val,'\n',model_choice,'\n',initial_sub,'\n\n')
    predited_words=predict_next_word_using_initial_substring(textbox_val[0:len(textbox_val)-len(initial_sub)],model_choice,initial_sub)
    #print(predited_words)
    listbox.delete(0,END)
         
    for item in predited_words[0]:
        #print(item)
        if model_choice=='1':
            listbox.insert(END,ReplaceBrackets(item[0]))
        else:
            #print(item)
            listbox.insert(END,ReplaceBrackets(item))
    #print(TextBox.get('1.0',END))
    
def listIndexSelected(evt):
    print('3')
    
def populate_list():
    for key in biWordsSortedTuple:
        predictionList.insert(i,key[0])
        i+=1
    listValue=str(predictionList.get(predictionList.curselection()))    
        
        
initial_sub=''        
root = Tk(className='Urdu Word Predictor')
frame1 = Frame(root, width=500, height = 5000, background="white",borderwidth=1)

var = IntVar()
R1 = Radiobutton(frame1, text="ایک گرام ماڈل", variable=var, value=1,
                  command=radioButtonSelected,width=12,height=2)
R1.pack( anchor = W )

R2 = Radiobutton(frame1, text="دو گرام ماڈل", variable=var, value=2,
                  command=radioButtonSelected,width=12,height=2)
R2.pack( anchor = W)

R3 = Radiobutton(frame1, text="تین گرام ماڈل", variable=var, value=3,
                  command=radioButtonSelected,width=12,height=2)
R3.pack( anchor = W)
R4 = Radiobutton(frame1, text="انٹرپولیشن ماڈِل", variable=var, value=4,
                  command=radioButtonSelected,width=12,height=2)
R4.pack( anchor = W)
R1.invoke()

frame2 = Frame(root, background="black",borderwidth=1)




TextBox = Text(frame2,width=150,height=40,font=r"JameelNooriNastaleeq 12")
TextBox.tag_configure('tag-right',justify = "right")
TextBox.bind(sequence="<KeyRelease>",func=keyPressedInTextBox)
TextBox.pack( anchor = W)

frame3 = Frame(root, background="white",borderwidth=1)
label = Label(frame3,width=19)
label.config(text = "پیشن گوہی کھڑکی ")
label.pack(anchor=E)




listbox = Listbox(frame3,width=23,height = 30)
listbox.bind('<<ListboxSelect>>',listIndexSelected)
listbox.pack(anchor = W)

frame2.pack(side = LEFT, padx = 10,pady=10)
frame1.pack (side = TOP ,padx = 10,pady=10,fill = X)
frame3.pack(side = BOTTOM, padx = 10,pady=10)
R1.invoke()
#frame1.place(relx=.5, rely=.5, anchor="c")



listbox.insert(END, "a list entry")

    
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\mateen.ahmad\AppData\Local\Continuum\Anaconda3\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
  File "<ipython-input-19-b4ba66ab650d>", line 13, in radioButtonSelected
    keyPressedInTextBox('')
  File "<ipython-input-19-b4ba66ab650d>", line 19, in keyPressedInTextBox
    textbox_val=TextBox.get('1.0',END)
NameError: name 'TextBox' is not defined
